In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#from __future__ import print_function, absolute_import, unicode_literals, division
#import six
#from six.moves import (zip, filter, map, reduce, input, range)

import sys

sys.path.append('..')
import about
import pathcustomize

#import os
#import pathlib
#import pickle
#import platform
about.about()

import pandas as pd
import math
import random

import matplotlib.pyplot as plt
import prettyplotlib as ppl
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.path as path
from mpltools import style
from mpltools import layout
from statsmodels.distributions.empirical_distribution import ECDF


from waldo.conf import settings
from waldo.wio.experiment import Experiment
import waldo.metrics.report_card as report_card
import waldo.collider
import waldo.tape as tp
import waldo.metrics.step_simulation as ssim
#DATA_DIR = settings.LOGISTICS['filesystem_data']
style.use('ggplot')

Python 3.4.1 (default, Feb 28 2015 09:14:07) [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.56)] on darwin, Host: peters-MacBook-Air.local


/Users/peterwinter/.pyenv/versions/3.4.1/lib/python3.4/site-packages/mpltools/style/__init__.py:42: FutureWarning: 

    The style-sheet functionality in mpltools has been integrated into
    Matplotlib >= 1.4. This module will be removed in a future release.

    Note that style-sheets used by `matplotlib.style` use the standard
    Matplotlib rc-file syntax instead of the INI format used by `mpltools`.
    This mostly means un-quoting strings and changing '=' to ':'.


  """, FutureWarning)


# ALL FUNCTIONS

In [3]:
def bridge_gaps(experiment, graph, threshold=0.001):
    taper = tp.Taper(experiment=experiment, graph=graph)
    start, end = taper.find_start_and_end_nodes()
    gaps = taper.score_potential_gaps(start, end)
    gt = taper.greedy_tape(gaps, threshold=threshold, add_edges=True)
    graph = taper._graph
    


In [4]:
# def step_plot(ax, step_df, true_num=30, only_best=False):

#     steps = []
#     n_steps = len(step_df)

#     xs = list(step_df['t0'])
#     widths = list(step_df['lifespan'])
#     height = 1

#     color_cycle = ax._get_lines.color_cycle
#     color1 = color_cycle.next()
#     color2 = color_cycle.next()
#     for y, (x, width) in enumerate(zip(xs, widths)):
#         steps.append(patches.Rectangle((x,y), height=height, width=width,
#                                        fill=True, fc=color1, ec=color1))
#     for step in steps:
#         ax.add_patch(step)

#     xmax = 60
#     ideal = patches.Rectangle((0,0), height=true_num, width=xmax,
#                               fill=True, ec=color2, fc=color2,
#                               alpha=0.5)

#     ax.add_patch(ideal)
#     ax.plot([0], color=color1, label='actual')
#     ax.plot([0], color=color2, alpha=0.5, label='ideal')
#     ax.set_xlim([0, xmax])
#     ax.set_ylim([0, n_steps+1])

def plot_steps(ax, df, dividers=[]):
    label = 'hi'
    step_df = df
    n_steps = len(step_df)    
    xmax = int(max(df['tN']))
    ymax = n_steps + 1
    steps = []
    

    xs = list(step_df['t0'])
    widths = list(step_df['lifespan'])
    height = 1

    color = ax._get_lines.color_cycle.next()
    for y, (x, width) in enumerate(zip(xs, widths)):
        steps.append(patches.Rectangle((x,y), height=height, width=width,
                                       fill=True, fc=color, ec=color))
    for step in steps:
        ax.add_patch(step)
        
    for d in dividers:
        ax.plot([0, xmax], [d,d], color='black')
        
    ax.plot([0], color=color, label=label)
    ax.set_xlim([0, xmax])
    ax.set_ylim([0, ymax])
    ax.set_xlabel('t (min)')
    
def make_dividers(step_dfs):
    dividers = [len(i) for i in step_dfs]
    dividers = [0] + dividers
    return np.cumsum(dividers)  

def step_plot(ax, df):
    a = df.sort('lifespan', ascending=False)
    a['mid'] = (a['tN'] + a['t0'])/2
    xmax = int(max(a['tN']))

    
    short_is_less_than = 5 
    long_is_greater_than = 40
    front_back_margin = 10
    
    front_lim = front_back_margin
    back_lim = xmax - front_back_margin

    #shortest = a[a['lifespan'] <= short_is_less_than]
    #a = a[a['lifespan'] > short_is_less_than]
    #shortest.sort('tN', inplace=True, ascending=True)
    #shortest.sort('t0', inplace=True, ascending=False)


    #longest = a[a['lifespan'] >= long_is_greater_than]
    #a = a[a['lifespan'] < long_is_greater_than]
    #longest.sort('t0', inplace=True)
    #longest.sort('tN', inplace=True, ascending=False)

    front = a[a['t0'] <= front_lim]
    a = a[a['t0'] > front_lim]
    front.sort('tN', inplace=True, ascending=False)


    back = a[a['tN'] >= back_lim]
    a = a[a['tN'] < back_lim]
    back.sort('t0', inplace=True, ascending=False)


    mid = a
    mid.sort('mid', inplace=True, ascending=False)

    
    steps = [front, mid, back]
    a = pd.concat(steps)
    div = make_dividers(steps)
    plot_steps(ax,a, dividers=div)

In [5]:
def calculate_duration_data_from_graph(experiment, graph, node_ids=[]):
    if not node_ids:
        node_ids = graph.nodes(data=False)
    
    frame_times = experiment.frame_times
    step_data, durations = [], []
    for node in node_ids:
        node_data = graph.node[node]
        bf, df = node_data['born_f'], node_data['died_f']
        t0 = frame_times[bf - 1]
        tN = frame_times[df - 1]
        step_data.append({'bid':node, 't0':t0, 'tN':tN, 'lifespan':tN-t0})

    steps = pd.DataFrame(step_data)
    steps.set_index('bid', inplace=True)
    steps = steps / 60.0   # convert to minutes.
    steps.sort('t0', inplace=True)
    steps = steps[['t0', 'tN', 'lifespan']]
    durations = np.array(steps['lifespan'])
    return steps, durations


In [6]:
def make_cdf(ax, durations_list, label='', xmin=0, xmax=30):
    x = np.linspace(xmin, xmax, 1000)
    ecdf = ECDF(np.array(durations_list))
    cdf = ecdf(x)
    ax.plot(x, cdf, label=label, lw=2)
    ax.set_xlabel('minutes')
    ax.set_ylabel('CDF')


In [7]:
def find_nodes_containing_bids(graph, bids):
    nodes_with_moved_bids = []
    for node in graph:
        if node in bids:
            nodes_with_moved_bids.append(node)
            continue
        for c in graph.node[node].get('components', []):
            if c in bids:
                nodes_with_moved_bids.append(node)
                break
    return nodes_with_moved_bids

In [8]:
def show_reasons(df):
    reasons = ['unknown', 'on_edge', 'id_change', 'outside-roi', 'timing']
    if 'reason' not in df:
        df['reason'] = 'unknown'
        for reason in reasons[1:]:
            df['reason'][df[reason]] = reason
            print reason, len(df['reason'][df[reason]])

    counts = {}
    for reason in reasons:
         counts[reason] = len(df[df['reason'] == reason])
            
    unknown_ids = list(df[df['reason'] == 'unknown'].index)
    id_change_ids = list(df[df['reason'] == 'id_change'].index)

    return unknown_ids, id_change_ids

SyntaxError: invalid syntax (<ipython-input-8-0f305f2cc47c>, line 7)

In [9]:
def rc_duration_hist_single(rc, step='roi'):
    rc = rc.copy()
    
    # grab bins from column names
    cols = rc.columns
    bins = [(int(c[1:-3]), c) for c in cols if str(c[0]) == '>']
    bins.sort(reverse=True)
    #print bins
    
    # grab relevant step
    step_data = rc.iloc[step]
    print step_data[['step', '>10min','>20min','>30min','>40min','>50min']]
    bin_data = []
    running_total = 0
    for bl, rc_label in bins:
        a = step_data[rc_label]
        n = a - running_total
        running_total += n
        #print bl, a, n
        #print 'total', running_total
        bin_data.append((bl, n))
    bin_data.sort()
    print bin_data
    fig, ax = plt.subplots()
    x = np.arange(len(bin_data))
    labels, y = zip(*bin_data)
    labels = [str(l) for l in labels]
    ppl.bar(ax, x, y, annotate=True, xticklabels=labels, grid='y')
    plt.show()

def rc_duration_hist(rc):
    rc = rc.copy()
    cols = rc.columns
    
    bins = [(int(c[1:-3]), c) for c in cols if str(c[0]) == '>']

    # grab bins from column names

    bins.sort(reverse=True)
    #print bins
    
    def step_data_to_bins(step):
        # grab relevant step
        step_data = rc.iloc[step]
        #print step_data[['step', '>10min','>20min','>30min','>40min','>50min']]
        bin_data = []
        running_total = 0
        for bl, rc_label in bins:
            a = step_data[rc_label]
            n = a - running_total
            running_total += n
            #print bl, a, n
            #print 'total', running_total
            bin_data.append((bl, n))
        bin_data.sort()
        bd = {} #'step':step_name}
        for l, n in bin_data:
            bd[l] = n
        return bd
    
    bd1 = step_data_to_bins(1)
    bd2 = step_data_to_bins(-1)
    
    #bd1['step'] = 'raw'
    #bd2['step'] = 'final'
    
    df = pd.DataFrame([bd1, bd2], index=['raw', 'final'])
    df.rename(columns={10:'10-19',20:'20-29', 
                       30:'30-39',40:'40-49', 50:'50-60'}, inplace=True)
 
    ax = df.T.plot(kind='bar')
    fig = plt.figure(plt.get_fignums()[0])
    fig.set_size_inches(10, 5)
    ax.set_ylabel('number of tracks')
    ax.set_xlabel('track duration (min)')
    plt.tight_layout()

    plt.savefig('track-len-improvement.png')

    #plt.show()
    #print df
    
    
    
#rc_duration_hist(rc)
#  


# Generate Report Card

In [10]:
# N = 15
ex_id = '20141017_113435'
ex_id = '20141017_113439'
ex_id = '20141017_123722'
ex_id = '20141017_123725'
# N = 25
#ex_id = '20141017_134720'
#ex_id = '20141017_134724'
ex_id = '20141017_150959'
#ex_id = '20141017_151002'

In [9]:
#ex_id = '20130614_120518'
#ex_id = '20130318_131111'
#ex_id = '20130702_135704' # testset
#ex_id = '20130702_135652'
#ex_id = '20130414_140704'
#ex_id = '20130410_165326'
ex_id ='20130324_155127'
ex_id ='20141017_123725'


ex_df = '20130318_131056' # from tests for making behavior movies
experiment = Experiment(experiment_id=ex_id)
original_graph = experiment.graph
graph = original_graph.copy()

MWTSummaryError: Could not find summary file in target path: /Users/peterwinter/Data/waldo/20141017_123725

In [11]:
experiment.directory

PosixPath('/home/projects/worm_movement/Data/MWT_RawData/20141017_123725')

In [12]:
solver = report_card.WaldoSolver(experiment=experiment, graph=graph)
graph2, rc = solver.run()

HELTENA: Reportcard init 0


IOError: File /home/visitors/peterwinter/codes/waldo/data/test/20141017_123725/waldo/20141017_123725-bounds.csv does not exist

In [ ]:
rc_duration_hist(rc)

In [ ]:
def node_summary(experiment, graph, node_ids=[]):
    """
    returns a dataframe with lots of data for every node specified in node_ids.
    
    dataframe columns are:
    'bl', 'components', 'f0', 'fN', 't0', 'tN',
    'x_max', 'x_min', 'y_max', y_min'
    
    params
    -----
    experiment: (wio.experiment object)
    node_ids: (list)

    """
    if not node_ids:
        node_ids = graph.nodes(data=False)
    
    frame_times = experiment.frame_times
    node_movement = graph2.node_movement(experiment)
    node_movement.sort(inplace=True)
    node_summaries = []
    for node in node_ids:
        node_data = graph.node[node]
        bf, df = node_data['born_f'], node_data['died_f']
        t0 = frame_times[bf - 1]
        tN = frame_times[df - 1]
        comps = list(node_data.get('components', [node]))
        comp_string = '-'.join([str(c) for c in comps])
        n_summary = {'bid':node, 'f0':bf, 'fN':df, 't0':t0, 'tN':tN, 'components':comp_string}
        if node in node_movement.index:
            n_move = node_movement.loc[node]
            for i, v in zip(n_move.index, n_move):
                n_summary[str(i)] = v   
        else:
            for i in ['x_max', 'x_min', 'y_max', 'y_min', 'bl']:
                n_summary[i] = 0
        node_summaries.append(n_summary)
    node_info = pd.DataFrame(node_summaries)
    node_info.set_index('bid', inplace=True)
    node_info.sort(inplace=True)
    return node_info
node_summary(experiment, graph2)    

## Graph how iterations reduce node count

In [ ]:
l = rc.set_index('step').loc[['iter 0', 'iter 1', 'iter 2', 'iter 3']][['total-nodes', 'moving-nodes']]
l.reset_index(inplace=True)
l.rename(columns={'total-nodes':'total tracks', 
                  'moving-nodes': 'move > 2 BL'})
l.index.name = 'iteration'
l[['total-nodes', 'moving-nodes']].plot(kind='bar')
l

In [ ]:
def rc_worm_minutes_hist(rc):
    rc = rc.copy()
    rc.rename(columns={'wm_0min':'0-9', 'wm_10min':'10-19','wm_20min':'20-29', 
                       'wm_30min':'30-39','wm_40min':'40-49', 'wm_50min':'50-60'}, inplace=True)
    rc = rc[['0-9', '10-19', '20-29', '30-39', '40-49', '50-60']]

    # for sanity check
    #totals = rc['0'] +  rc['10'] + rc['20'] + rc['30'] + rc['40'] + rc['50']
    #print totals
    # grab bins from column names
    cols = rc.columns
    bins = [(int(c[1:-3]), c) for c in cols if str(c[0]) == '>']
    bins.sort(reverse=True)
    #print bins
    
    def step_data_to_bins(step):
        # grab relevant step
        step_data = rc.iloc[step]
        #step_name = step_data['step']
        
        return step_data
    
    bd1 = step_data_to_bins(1)
    bd2 = step_data_to_bins(-1)
    
    #bd1['step'] = 'raw'
    #bd2['step'] = 'final'
    
    df = pd.DataFrame([bd1, bd2], index=['raw', 'final'])
    #df.set_index('step', inplace=True)
    ax = df.T.plot(kind='bar')
    fig = plt.figure(plt.get_fignums()[0])
    fig.set_size_inches(10, 5)

    ax.set_ylabel('total minutes of track')
    ax.set_xlabel('track duration (min)')
    plt.savefig('worm-min-improvement.png')
    plt.tight_layout()
    #plt.show()
    print df
    
rc_worm_minutes_hist(rc)

In [ ]:
experiment.prepdata.load('end_report')

# step graphs for moving nodes

check if nodes that moved in raw data are still in the final graph

In [ ]:
min_move = 1

moved_bids = [int(i) for i in graph.compound_bl_filter(experiment, threshold=min_move)]
new_moved_bids = [int(i) for i in graph2.compound_bl_filter(experiment, threshold=min_move)]

print len(moved_bids), 'nodes moved over threshold in origional graph'
print len(new_moved_bids), 'nodes moved over threshold in final graph'

Create step plots for before and after we 'fix' the data

In [ ]:
true_num = experiment.true_num

steps0, durations0 = calculate_duration_data_from_graph(experiment, original_graph, original_graph.nodes(data=False))
steps, durations = calculate_duration_data_from_graph(experiment, original_graph, moved_bids)

fig, ax = plt.subplots()
step_plot(ax, steps)
ax.set_title('original blobs moving > {t} BL'.format(t=min_move))
ax.set_xlabel('minutes')
plt.show()

steps2, durations2 = calculate_duration_data_from_graph(experiment, graph2, graph2.nodes(data=False))

fig, ax = plt.subplots()
step_plot(ax, steps2)
ax.set_title('all blobs in final graph')
ax.set_xlabel('minutes')
plt.show()

moved_steps2, moved_durations2 = calculate_duration_data_from_graph(experiment, graph2, new_moved_bids)

fig, ax = plt.subplots()
step_plot(ax, moved_steps2)
ax.set_title('final blobs moving > {t} BL'.format(t=min_move))
ax.set_xlabel('minutes')
plt.show()

In [ ]:
df = ssim.run_ideal_step_simulation(experiment, verbose=True)
fig, ax = plt.subplots()
step_plot(ax, df, true_num=experiment.true_num)

In [ ]:
def top_step_plot(ax, step_df, min_duration=10, true_num=30):
    best = step_df[step_df['lifespan'] >= 10]
    print best.head()
    
    steps = []
    n_steps = len(best)

    xs = list(best['t0'])
    widths = list(best['lifespan'])
    height = 1

    color_cycle = ax._get_lines.color_cycle
    color1 = color_cycle.next()
    color2 = color_cycle.next()
    for y, (x, width) in enumerate(zip(xs, widths)):
        steps.append(patches.Rectangle((x,y), height=height, width=width,
                                       fill=True, fc=color1, ec=color1))
    for step in steps:
        ax.add_patch(step)

    xmax = 60
    ideal = patches.Rectangle((0,0), height=true_num, width=xmax,
                              fill=True, ec=color2, fc=color2,
                              alpha=0.5)

    ax.add_patch(ideal)
    ax.plot([0], color=color1, label='actual')
    ax.plot([0], color=color2, alpha=0.5, label='ideal')
    ax.set_xlim([0, xmax])
    ax.set_ylim([0, n_steps+1])

steps0, durations0 = calculate_duration_data_from_graph(experiment, original_graph, original_graph.nodes(data=False))
fig, ax = plt.subplots()
top_step_plot(ax, steps0)
ax.set_title('blobs > 10 min in raw data')
ax.set_xlabel('minutes')
plt.show()

steps2, durations2 = calculate_duration_data_from_graph(experiment, graph2, graph2.nodes(data=False))
fig, ax = plt.subplots()
top_step_plot(ax, steps2)
ax.set_title('blobs > 10 min in final data')
ax.set_xlabel('minutes')
plt.show()

In [ ]:
def compare_step_plots(step_df1, step_df2, titles=['',''], true_num=30, i=0):

    
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(20, 10)

    #steps = []
    n_steps = max(len(step_df1), len(step_df2))
    xmax = 60
    def make_steps(ax, df, i=1):
        steps = []
        xs = list(df['t0'])
        widths = list(df['lifespan'])
        height = 1

        color_cycle = ax._get_lines.color_cycle
        color1 = color_cycle.next()
        color2 = color_cycle.next()
        for y, (x, width) in enumerate(zip(xs, widths)):
            if i == 0:
                c = color1
            else:
                c = color2
            steps.append(patches.Rectangle((x,y), height=height, width=width,
                                           fill=True, fc=c, ec=c))
        for step in steps:
            ax.add_patch(step)


        ideal = patches.Rectangle((0,0), height=true_num, width=xmax,
                                  fill=True, ec=color2, fc=color2,
                                  alpha=0.5)

        ax.add_patch(ideal)
        #if i != 2:
        #    ax.plot([0], color=color1, label='actual')
        #else:
        #ax.plot([0], color=color1, label='actual')
        #ax.plot([0], color=color2, alpha=0.5, label='ideal')
        ax.set_xlim([0, xmax])
        ax.set_ylim([0, n_steps+1])
        ax.set_xlabel('minutes')
        ax.set_ylabel('tracks')
        
    make_steps(ax1, step_df1, i=0)
    ax1.set_title(titles[0])
    make_steps(ax2, step_df2, i=2)
    ax2.set_title(titles[1])

    for ax in [ax1, ax2]:
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(20)
    plt.savefig('final-step-plot.png')
titles = ['raw (blobs >10 min)', 'final (blobs >10 min)']
compare_step_plots(steps0[steps0['lifespan'] >= 10], steps2[steps2['lifespan'] >= 10], titles=titles)

In [ ]:

def compare_step_plots2(step_df1, step_df2, true_num=30):

    
    fig, ax = plt.subplots()
    #fig.set_size_inches(20, 10)
    
    #steps = []
    n_steps = max(len(step_df1), len(step_df2))
    xmax = 60
    def make_steps(ax, df, alpha=1.0):
        steps = []
        xs = list(df['t0'])
        widths = list(df['lifespan'])
        height = 1

        color_cycle = ax._get_lines.color_cycle
        color1 = color_cycle.next()
        color2 = color_cycle.next()
        for y, (x, width) in enumerate(zip(xs, widths)):
            steps.append(patches.Rectangle((x,y), height=height, width=width,
                                           fill=True, fc=color1, ec=color1, alpha=alpha))
        for step in steps:
            ax.add_patch(step)


        #ideal = patches.Rectangle((0,0), height=true_num, width=xmax,
        #                          fill=True, ec=color2, fc=color2,
        #                          alpha=0.5)

        #ax.add_patch(ideal)
        #ax.plot([0], color=color1, alpha=alpha)
        ax.set_xlim([0, xmax])
        ax.set_ylim([0, n_steps+1])
        ax.set_xlabel('minutes')
        ax.set_ylabel('tracks')
        
    make_steps(ax, step_df1)
    make_steps(ax, step_df2, alpha=0.5)
    
titles = ['blobs >10 min from raw data', 'blobs >10 min from final data']
compare_step_plots(steps0[steps0['lifespan'] >= 10], steps2[steps2['lifespan'] >= 10], titles=titles)
#compare_step_plots2(steps0[steps0['lifespan'] >= 10], steps2[steps2['lifespan'] >= 10])

#compare_step_plots(steps2[steps2['lifespan'] >= 10], steps0[steps0['lifespan'] >= 10])
plt.show()

create CDF showing the duration difference

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 8)
make_cdf(ax, [d for d in durations0 if d>10], label= 'original nodes > 10 min', xmin=10, xmax=60)
make_cdf(ax, [d for d in durations2 if d>10], label='final nodes > 10 min', xmin=10, xmax=60)
ax.legend(loc='lower right', prop={'size':20})

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
         ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)
#plt.show()
plt.savefig('long-cdf-improvement.png')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 8)
make_cdf(ax, durations0, label= 'all original', xmax=30)
make_cdf(ax, durations2, label='all final nodes', xmax=30)
ax.legend(loc='lower right', prop={'size':20})
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
         ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)
#plt.show()
plt.savefig('all-cdf-improvement.png')

In [ ]:
fig, ax = plt.subplots()
make_cdf(ax, durations0, label= 'all original', xmax=60)
make_cdf(ax, durations2, label='all final nodes', xmax=60)
make_cdf(ax, [d for d in durations0 if d>10], label= 'original nodes > 10 min', xmin=10, xmax=60)
make_cdf(ax, [d for d in durations2 if d>10], label='final nodes > 10 min', xmin=10, xmax=60)
ax.legend(loc='lower right')
plt.show()

In [ ]:
fig, ax = plt.subplots()
make_cdf(ax, durations0, label= 'all original')
make_cdf(ax, durations, label= 'original > 2 BL')
make_cdf(ax, durations2, label='all final nodes')
make_cdf(ax, moved_durations2, label='final nodes that origionally moved > 2 BL')
ax.legend(loc='lower right')
plt.show()

TODO: create plots showing how applying the same threshold to our data before and after fixing it
    changes our True Positive / False Positives / False Negative rates

In [ ]:
#TODO format this to work for graph based data.
def calculate_stats_for_bid_lists(bid_lists, prep_data):
    
    base_accuracy = prep_data.load('accuracy')
    matches = prep_data.load('matches')
    counts, tps, fps, fns = [], [], [], []
    for bids in bid_lists:
        filtered_accuracy = ea.recalculate_accuracy(matches, base_accuracy, bids=bids)
        counts.append(len(bids))
        tp = filtered_accuracy['true-pos'].mean()
        fp = filtered_accuracy['false-pos'].mean()
        fn = filtered_accuracy['false-neg'].mean()
        tps.append(tp)
        fps.append(fp)
        fns.append(fn)

    tps = np.array(tps)
    fps = np.array(fps)
    fns = np.array(fns)
    totals = fns + tps
    tps_p = tps / totals * 100
    fps_p = fps / totals * 100
    fns_p = fns / totals * 100

    print('true counts=', np.mean(totals))

    data = pd.DataFrame([tps_p, fns_p, fps_p], index=['TP', 'FN', 'FP']).T
    counts = pd.DataFrame(counts, columns=['counts'])
    return data, counts

# Error Checking Cells

In [ ]:
unknown_ids, id_change_ids = show_reasons(ends)
#print unknown_ids
#print id_change_ids
unk = pd.DataFrame(unknown_ids)
name = '{eid}-unknown_ids.csv'.format(eid=ex_id)
print name
unk.to_csv(name, index=False, header=False)

col = pd.DataFrame(id_change_ids)
name = '{eid}-unresolved_collision_ids.csv'.format(eid=ex_id)
col.to_csv(name, index=False, header=False)


In [ ]:
us = starts[starts['reason'] == 'unknown']
ue = ends[ends['reason'] == 'unknown'].reset_index()
#print us.head()
ue = ue.rename(columns={'node_id':'node1', 'bid':'blob1'})
ue = ue[['blob1', 'node1', 't', 'x', 'y']]
#print ue.head()

match_data = []
for blob2, s in starts.iterrows():
    node2 = s['node_id']
    x, y = s['x'], s['y']
    t = s['t']
    
    print blob2, node2, x, y, t
    match_check = ue.copy()
    match_check['node2'] = node2
    match_check['blob2'] = node2
    match_check['dt'] = t - match_check['t']
    match_check['dx'] = np.fabs(x - match_check['x'])
    match_check['dy'] = np.fabs(y - match_check['y'])
    match_check['dist'] = np.sqrt(match_check['dt']**2 + match_check['dx']**2 + match_check['dy']**2)
    match_check['d_spacetime'] = np.sqrt(match_check['dt']**2 + match_check['dx']**2 + match_check['dy']**2)

    match_check = match_check[['blob1', 'blob2', 'dt', 'dist', 'd_spacetime']]
    match_check.sort('d_spacetime', inplace=True)
    print match_check.head()
    
    



these are two cells that aren't part of the workflow but that were used to sanity check the processing

In [ ]:
def double_check_duration(experiment, graph1, graph2, node_ids):
    frame_times = experiment.frame_times
    step_data, durations = [], []
    good_nodes = []
    bad_nodes = []
    for node in node_ids:
        node_data = graph2.node[node]
        bf, df = node_data['born'], node_data['died']
        components = node_data.get('components', [])
        
        node_checks_out = True
        for c in components:
            nd = graph1.node[c]
            cb, cd = nd['born'], nd['died']
            if cb < bf:
                node_checks_out = False
            if cd > df:
                node_checks_out = False
        if node_checks_out:
            good_nodes.append(node)
        else:
            bad_nodes.append(node)
    print len(bad_nodes), 'bad nodes'
    print len(good_nodes), 'good nodes'
                

double_check_duration(experiment, original_graph, graph2, graph2.nodes(data=False))

In [ ]:
print len(moved_bids), 'nodes moved over threshold in origional graph'
graph3 = original_graph.copy()
print 'orig', len(find_nodes_containing_bids(graph3, moved_bids))
collider.remove_nodes_outside_roi(graph3, experiment)
print 'roi', len(find_nodes_containing_bids(graph3, moved_bids))
collider.remove_blank_nodes(graph3, experiment)
print 'blank', len(find_nodes_containing_bids(graph3, moved_bids))
collider.removal_suite(graph3) #, assimilate=-1)
print 'suite', len(find_nodes_containing_bids(graph3, moved_bids))
bridge_gaps(experiment, graph3, threshold=0.001)
print 'gaps', len(find_nodes_containing_bids(graph3, moved_bids))

steps3, durations3 = calculate_duration_data_from_graph(experiment, graph3, graph3.nodes(data=False))

fig, ax = plt.subplots()
step_plot(ax, steps3)
ax.set_title('second try at figuring out graph')
ax.set_xlabel('minutes')

fig, ax = plt.subplots()
make_cdf(ax, durations3)

plt.show()



In [ ]:
# WEIRD!
# old and new functions for detecting movement do not match up 100%
# new function uses generalized body lenght, old one uses individual body length ...

min_move = 3

move = experiment.prepdata.load('moved')
moved_bids = list(set(move[move['bl_moved'] >= min_move]['bid']))
moving_nodes = [int(i) for i in graph.compound_bl_filter(experiment, threshold=min_move)]
print len(moved_bids), 'nodes moved over threshold in origional graph'
print len(moving_nodes), 'found with modern methods'
print len(set(moved_bids) & set(moving_nodes))
print len(find_nodes_containing_bids(original_graph, moved_bids)), 'check. this number should be same as first'